# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Passo 1: Pegar a cotaçao do dolar
navegador = webdriver.Chrome()

#entrou google
navegador.get("https://www.google.com/")
#achou um elemento na pag
navegador.find_element("xpath","/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotacao dolar")
navegador.find_element("xpath","/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]").send_keys(Keys.ENTER)
#pegando a cotacao
cotacao_dolar = navegador.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

#2 == cotacao do euro
navegador.get("https://www.google.com/")
navegador.find_element("xpath","/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotacao euro")
navegador.find_element("xpath","/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]").send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#3 == cotacao do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

navegador.quit()

5.1728
5.2900673760000005
293.37


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [18]:
#Passo 4: Atualizar a base dados 
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [23]:
#Passo 5: Recalcular os preços 

#atualizar as cotacoes
tabela.loc[tabela["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)


#preco de compra = preco original x cotacao
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]


#preco de venda = preco de compra x margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.172800,5172.748272,1.40,7241.847581
1,Carro Renault,4500.00,Euro,5.290067,23805.303192,2.00,47610.606384
2,Notebook Dell,899.99,Dólar,5.172800,4655.468272,1.70,7914.296062
3,IPhone,799.00,Dólar,5.172800,4133.067200,1.70,7026.214240
4,Carro Fiat,3000.00,Euro,5.290067,15870.202128,1.90,30153.384043
5,Celular Xiaomi,480.48,Dólar,5.172800,2485.426944,2.00,4970.853888
6,Joia 20g,20.00,Ouro,293.370000,5867.400000,1.15,6747.510000


### Agora vamos exportar a nova base de preços atualizada

In [25]:
#Passo 6: Exportar a base de dados
tabela.to_excel("Produtos Novo.xlsx",index=False)